In [2]:
import pandas as pd
import numpy as np

In [17]:
rf_all = pd.read_csv('Eve_Metrics/random_forest_all_fea.csv')
rf_corr = pd.read_csv('Eve_Metrics/random_forest_corr_fea.csv')
xgb_all = pd.read_csv('Eve_Metrics/xgb_all_fea.csv')
xgb_corr = pd.read_csv('Eve_Metrics/xgb_corr_fea.csv')
gbr_all = pd.read_csv('Eve_Metrics/gbr_all_fea.csv')
gbr_corr = pd.read_csv('Eve_Metrics/gbr_corr_fea.csv')

rf_all.shape, rf_corr.shape, xgb_all.shape, xgb_corr.shape, gbr_all.shape, gbr_corr.shape

((1, 4), (1, 4), (1, 4), (1, 4), (1, 4), (1, 4))

In [54]:
#Need to normalize each of these scores by transposint the dfs
from sklearn.preprocessing import MinMaxScaler

By normalizing each evaluation metric, we can compare and assess which one is better since every value is under a uniform scale

In [52]:
def normalizeEve(df):
    scaler = MinMaxScaler(feature_range=(0, 1))

    df_t = df.T.reset_index()
    df_t.rename(columns={'index': 'metric', 0: 'value'}, inplace=True)
    df_t['norm'] = scaler.fit_transform(df_t[['value']])
    
    df_temp = df_t.T.reset_index()
    df_temp.drop('index', axis=1, inplace=True)
    df_temp.columns = df_temp.iloc[0]
    df_temp.drop(df_temp.index[[0,1]], inplace=True)
    return df_temp

In [53]:
rf_all_norm = normalizeEve(rf_all)
rf_all_norm

,R2_Score,RMSE,MAE,MAPE
2,0.095098,1.0,0.72905,0.0


In [55]:
rf_corr_norm = normalizeEve(rf_corr)
xgb_all_norm = normalizeEve(xgb_all)
xgbcorr_norm = normalizeEve(xgb_corr)
gbr_allr_norm = normalizeEve(gbr_all)
gbr_corr_norm = normalizeEve(gbr_corr)

In [60]:
def evaluateM(rf, xgb, gbr):
    cols = rf.columns

    for i in cols:
        if i == 'R2_Score':
            if (rf[i] > xgb[i]).all():
                if (rf[i] > gbr[i]).all():
                    print('Random Forest model has highest R2 Score')
                else:
                    print('GBR has the highest R2 Score')
            elif (xgb[i] > gbr[i]).all():
                print('XGB has the highest R2 Score')
            else:
                print('GBR has the highest R2 Score')
        else:
            if (rf[i] < xgb[i]).all():
                if (rf[i] < gbr[i]).all():
                    print(f'Random Forest model has lowest {i}')
                else:
                    print(f'GBR has the lowest {i}')
            elif (xgb[i] < gbr[i]).all():
                print(f'XGB has the lowest {i}')
            else:
                print(f'GBR has the lowest {i}')

In [61]:
#Evaluating the all features of each model
evaluateM(rf_all_norm, xgb_all_norm, gbr_allr_norm)

XGB has the highest R2 Score
XGB has the lowest RMSE
Random Forest model has lowest MAE
GBR has the lowest MAPE


In [62]:
evaluateM(rf_corr_norm, xgbcorr_norm, gbr_corr_norm)

GBR has the highest R2 Score
GBR has the lowest RMSE
GBR has the lowest MAE
GBR has the lowest MAPE


This suggests that gradient boost model is better when only selected with the positively correlating features

On the other hand when all features are taken into consideration, XG Boost shows the best performance compared to the other 2 models.

We can choose gradient boost model as it has better results in both scenarios compared to other 2 models